In [1]:
import os
import xml.etree.ElementTree as ET

In [2]:
# Paths
annotations_dir = "../data/annotations"
output_dir = "../data/labels"

In [3]:
# Define class mapping
class_mapping = {
    "With Helmet": 0,
    "Without Helmet": 1
}

In [4]:
# Loop through all XML files
for xml_file in os.listdir(annotations_dir):
    if not xml_file.endswith(".xml"):
        continue

    tree = ET.parse(os.path.join(annotations_dir, xml_file))
    root = tree.getroot()

    # Get image size
    size = root.find("size")
    img_w = int(size.find("width").text) # type: ignore
    img_h = int(size.find("height").text) # type: ignore

    # Prepare YOLO lines
    yolo_lines = []
    for obj in root.findall("object"):
        class_name = obj.find("name").text # type: ignore
        if class_name not in class_mapping:
            continue  # skip unknown classes

        class_id = class_mapping[class_name]

        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text) # type: ignore
        ymin = int(bndbox.find("ymin").text) # type: ignore
        xmax = int(bndbox.find("xmax").text) # type: ignore
        ymax = int(bndbox.find("ymax").text)# type: ignore

        # Convert to YOLO format
        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h

        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # Save to .txt file with same name as image
    txt_file = xml_file.replace(".xml", ".txt")
    with open(os.path.join(output_dir, txt_file), "w") as f:
        f.write("\n".join(yolo_lines))